<a href="https://colab.research.google.com/github/anash18/4105-HW-Nash/blob/main/ECGR_4105_HW5/ECGR_4105_HW5_Problem_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.optim as optim

In [43]:
#Calling the dataset
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/4105 Datasets/Housing.csv'
housing = pd.DataFrame(pd.read_csv(file_path))
housing.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [44]:
#Applying a binary map
#mapping the variables to a binary output
varlist =  ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus']

# Defining the map function
def mapping(x):
    return x.map({'yes': 1, 'no': 0, 'furnished':  1, 'semi-furnished':  0, 'unfurnished':  -1})

housing[varlist] = housing[varlist].apply(mapping)

housing.head()
#housing.dtypes

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,1
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,1
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,0
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,1
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,1


In [45]:
#Splitting the Dataset
x3_train, x3_val = train_test_split(housing, train_size = 0.8, test_size = 0.2, random_state = 0)
x3_train.head()

#Splitting Variables
y3_train = x3_train.pop("price")
y3_val = x3_val.pop("price")

#print(x3_train)
#print(y3_train)
#x3_train.head()
#print(y3_val)

In [46]:
#Standardizing the Dataset

standard = StandardScaler()

vars3 = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking', 'mainroad', 'guestroom', 'basement', 'hotwaterheating',
         'airconditioning', 'prefarea', 'furnishingstatus']

x3_train_stan = x3_train
x3_train_stan[vars3] = standard.fit_transform(x3_train_stan[vars3])

x3_val_stan = x3_val
x3_val_stan[vars3] = standard.fit_transform(x3_val_stan[vars3])
#x3_val_stan.head()

In [47]:
#Stacking them there inputs
x3_train_stan_numpy = np.c_[np.ones((len(y3_train), 1)), x3_train_stan[vars3]]
x3_val_stan_numpy = np.c_[np.ones((len(y3_val), 1)), x3_val_stan[vars3]]

#I am become tensor destroyer of deep learning
x3_train_ten = torch.tensor(x3_train_stan_numpy)
x3_val_ten = torch.tensor(x3_val_stan_numpy)
y3_train_ten = torch.tensor(y3_train.values)
y3_val_ten = torch.tensor(y3_val.values)


y3_train_mean = y3_train_ten.float().mean()
y3_train_std = y3_train.std()
y3_train_stan_ten = (y3_train_ten-y3_train_mean) / y3_train_std

y3_val_mean = y3_val_ten.float().mean()
y3_val_std = y3_val.std()
y3_val_stan_ten = (y3_val_ten-y3_val_mean) / y3_val_std

In [48]:
#Getting lost and starting my modeling career

#defining the model
def model(t_u, w12, w11, w10, w9, w8, w7, w6, w5, w4, w3, w2, w1, b):
  return w12*t_u**12 + w11*t_u**11 + w10*t_u**10 + w9*t_u**9 + w8*t_u**8 + w7*t_u**7 + w6*t_u**6 + w5*t_u**5 + w4*t_u**4 + w3*t_u**3 + w2 * t_u ** 2 + w1 * t_u + b

#defining the loss function
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

#Setting up the initial parameters
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)

In [49]:
#Time for an anime training arc

#defining a training loop
def training_loop(n_epochs, optimizer, params, t_u_train, t_c_train, t_u_val, t_c_val):
  for epoch in range (1, n_epochs +1):

    if params.grad is not None:  # <1>
      params.grad.zero_()

    t_p_train = model(t_u_train, *params)
    loss_train = loss_fn(t_p_train.transpose(0,1), t_c_train)

    t_p_val = model(t_u_val, *params)
    loss_val = loss_fn(t_p_val.transpose(0,1), t_c_val)

    optimizer.zero_grad()
    loss_val.backward()
    optimizer.step()

    if epoch <= 3 or epoch % 500 == 0:
            print(f"Epoch {epoch}, Training loss: {loss_train.item():.4f},"
                  f" Validation loss: {loss_val.item():.4f}\n")
  return params


#SGD Number 1 learning rate of 0.00000000000001
learning_rate_SGD1 = 0.00000000000001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
optimizer_SGD1 = optim.SGD([params], lr=learning_rate_SGD1)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_SGD1,
    params = params,
    t_u_train = x3_train_ten,
    t_c_train = y3_train_ten,
    t_u_val = x3_val_ten,
    t_c_val = y3_val_ten,
)

Epoch 1, Training loss: 296683442882877.3750, Validation loss: 54364782483269.1953

Epoch 2, Training loss: 116209106171211.4531, Validation loss: 34261826607544.3672

Epoch 3, Training loss: 57396814506533.7500, Validation loss: 27932703149257.7383

Epoch 500, Training loss: 26642197111321.9492, Validation loss: 25021440860438.3438

Epoch 1000, Training loss: 26629905782189.8867, Validation loss: 25018454628682.7539

Epoch 1500, Training loss: 26617971989249.1758, Validation loss: 25015514363280.1523

Epoch 2000, Training loss: 26606390379735.8594, Validation loss: 25012619310923.7031

Epoch 2500, Training loss: 26595152695986.5352, Validation loss: 25009768850515.4531

Epoch 3000, Training loss: 26584253424455.1094, Validation loss: 25006962244746.5898

Epoch 3500, Training loss: 26573686007767.4414, Validation loss: 25004198814493.0117

Epoch 4000, Training loss: 26563442712658.9805, Validation loss: 25001477890864.6758

Epoch 4500, Training loss: 26553518583133.3359, Validation los

tensor([-3.6225e-01,  1.7576e+00,  1.5245e+00,  1.1999e+00,  1.0805e+00,
         1.0254e+00,  1.0105e+00,  1.0030e+00,  1.0018e+00,  1.0006e+00,
         1.0006e+00,  1.0000e+00,  4.7060e-04], requires_grad=True)

In [50]:
#SGD Number 2 learning rate of 0.000000000000001
learning_rate_SGD2 = 0.000000000000001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
optimizer_SGD2 = optim.SGD([params], lr=learning_rate_SGD2)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_SGD2,
    params = params,
    t_u_train = x3_train_ten,
    t_c_train = y3_train_stan_ten,
    t_u_val = x3_val_ten,
    t_c_val = y3_val_stan_ten,
)

Epoch 1, Training loss: 282610389830254.3125, Validation loss: 33358261732576.9219

Epoch 2, Training loss: 258204261469765.4688, Validation loss: 30494463018883.6641

Epoch 3, Training loss: 235899816562740.5312, Validation loss: 27876531415463.8828

Epoch 500, Training loss: 13753132772.5855, Validation loss: 1387068287.8940

Epoch 1000, Training loss: 13730083027.7807, Validation loss: 1384913113.8427

Epoch 1500, Training loss: 13707052622.2765, Validation loss: 1382759617.7856

Epoch 2000, Training loss: 13684000966.2717, Validation loss: 1380607793.6122

Epoch 2500, Training loss: 13661009273.5006, Validation loss: 1378457653.5394

Epoch 3000, Training loss: 13637996398.5725, Validation loss: 1376309185.3516

Epoch 3500, Training loss: 13615043418.5345, Validation loss: 1374162401.2631

Epoch 4000, Training loss: 13592069324.6830, Validation loss: 1372017289.0609

Epoch 4500, Training loss: 13569155057.3781, Validation loss: 1369873860.9566

Epoch 5000, Training loss: 13546260129

tensor([-2.0432e-01,  7.4217e-01,  9.4415e-01,  9.8760e-01,  9.9721e-01,
         9.9925e-01,  9.9990e-01,  9.9998e-01,  1.0000e+00,  1.0000e+00,
         1.0000e+00,  1.0000e+00, -4.4248e-08], requires_grad=True)

In [51]:
#SGD Number 3 learning rate of 0.0000000000000001
learning_rate_SGD3 = 0.0000000000000001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
optimizer_SGD3 = optim.SGD([params], lr=learning_rate_SGD3)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_SGD3,
    params = params,
    t_u_train = x3_train_ten,
    t_c_train = y3_train_stan_ten,
    t_u_val = x3_val_ten,
    t_c_val = y3_val_stan_ten,
)

Epoch 1, Training loss: 282610389830254.3125, Validation loss: 33358261732576.9219

Epoch 2, Training loss: 280120190501501.7500, Validation loss: 33066099160987.6797

Epoch 3, Training loss: 277651873603160.5312, Validation loss: 32776496213211.3242

Epoch 500, Training loss: 3178503028434.4673, Validation loss: 415101933484.8976

Epoch 1000, Training loss: 14939092401.5714, Validation loss: 6474958774.4521

Epoch 1500, Training loss: 9599497837.0047, Validation loss: 1451106034.2778

Epoch 2000, Training loss: 13251960575.0564, Validation loss: 1389127740.5324

Epoch 2500, Training loss: 13706904691.3531, Validation loss: 1388143256.5508

Epoch 3000, Training loss: 13755280025.5556, Validation loss: 1387909062.9320

Epoch 3500, Training loss: 13757324936.2406, Validation loss: 1387677091.5278

Epoch 4000, Training loss: 13754852759.7497, Validation loss: 1387444580.3893

Epoch 4500, Training loss: 13752421495.4475, Validation loss: 1387212094.3031

Epoch 5000, Training loss: 13749949

tensor([-2.0616e-01,  7.5023e-01,  9.4809e-01,  9.8913e-01,  9.9779e-01,
         9.9954e-01,  9.9991e-01,  9.9999e-01,  1.0000e+00,  1.0000e+00,
         1.0000e+00,  1.0000e+00, -1.2297e-08], requires_grad=True)

In [52]:
#SGD Number 4 learning rate of 00.00000000000000001
learning_rate_SGD4 = 0.00000000000000001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
optimizer_SGD4 = optim.SGD([params], lr=learning_rate_SGD4)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_SGD4,
    params = params,
    t_u_train = x3_train_ten,
    t_c_train = y3_train_stan_ten,
    t_u_val = x3_val_ten,
    t_c_val = y3_val_stan_ten,
)

Epoch 1, Training loss: 282610389830254.3125, Validation loss: 33358261732576.9219

Epoch 2, Training loss: 282360873518377.5625, Validation loss: 33328987585144.8398

Epoch 3, Training loss: 282111581519872.0625, Validation loss: 33299739686347.0078

Epoch 500, Training loss: 181816136938991.0938, Validation loss: 21524931025692.5430

Epoch 1000, Training loss: 116782054222841.2656, Validation loss: 13877294401057.6934

Epoch 1500, Training loss: 74941516814466.5938, Validation loss: 8946958543863.7910

Epoch 2000, Training loss: 48036797924804.5391, Validation loss: 5768456767524.4600

Epoch 2500, Training loss: 30747255959146.9961, Validation loss: 3719325001433.6255

Epoch 3000, Training loss: 19645552597539.1641, Validation loss: 2398283632597.4971

Epoch 3500, Training loss: 12524260821422.7402, Validation loss: 1546630100679.4392

Epoch 4000, Training loss: 7962007827826.2344, Validation loss: 997581642224.1919

Epoch 4500, Training loss: 5043873188619.8916, Validation loss: 643

tensor([-7.2006e-02,  7.7874e-01,  9.5429e-01,  9.9055e-01,  9.9804e-01,
         9.9959e-01,  9.9995e-01,  1.0000e+00,  1.0000e+00,  1.0000e+00,
         1.0000e+00,  1.0000e+00, -8.1176e-09], requires_grad=True)

In [53]:
#Adam Number 1 learning rate of 0.1
learning_rate_Adam1 = 0.1
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
optimizer_Adam1 = optim.Adam([params], lr=learning_rate_Adam1)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_Adam1,
    params = params,
    t_u_train = x3_train_ten,
    t_c_train = y3_train_ten,
    t_u_val = x3_val_ten,
    t_c_val = y3_val_ten,
)

Epoch 1, Training loss: 296683442882877.3750, Validation loss: 54364782483269.1953

Epoch 2, Training loss: 244227160115067.0625, Validation loss: 48444982659405.3125

Epoch 3, Training loss: 197621788670653.6875, Validation loss: 43214574532601.0703

Epoch 500, Training loss: 26563288423066.9062, Validation loss: 24982959743090.7930

Epoch 1000, Training loss: 26334140750106.7383, Validation loss: 24883873310091.3242

Epoch 1500, Training loss: 26309421666252.1094, Validation loss: 24811513910579.6992

Epoch 2000, Training loss: 26435670925473.2891, Validation loss: 24772719941928.7031

Epoch 2500, Training loss: 26571895845521.0312, Validation loss: 24754708602261.8945

Epoch 3000, Training loss: 26630010767769.5430, Validation loss: 24743899777606.8750

Epoch 3500, Training loss: 26607879889250.7344, Validation loss: 24733499947127.6797

Epoch 4000, Training loss: 26539365153284.7305, Validation loss: 24721543078952.5391

Epoch 4500, Training loss: 26450125114062.4766, Validation lo

tensor([-4.9666e-01, -2.7005e+00,  1.5315e+01,  2.1785e+01,  1.0997e+02,
         1.4651e+02,  3.1509e+02,  3.4710e+02,  4.4475e+02,  4.5163e+02,
         4.9095e+02,  4.7979e+02,  4.9908e+02], requires_grad=True)

In [54]:
#Adam Number 2 learning rate of 00.01
learning_rate_Adam2 = 0.01
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
optimizer_Adam2 = optim.Adam([params], lr=learning_rate_Adam2)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_Adam2,
    params = params,
    t_u_train = x3_train_ten,
    t_c_train = y3_train_ten,
    t_u_val = x3_val_ten,
    t_c_val = y3_val_ten,
)

Epoch 1, Training loss: 296683442882877.3750, Validation loss: 54364782483269.1953

Epoch 2, Training loss: 291183471270045.6250, Validation loss: 53742780748021.2656

Epoch 3, Training loss: 285741603462669.7500, Validation loss: 53127629423985.5938

Epoch 500, Training loss: 26793621066594.4961, Validation loss: 25055069063528.4648

Epoch 1000, Training loss: 26771810141193.9375, Validation loss: 25048605007804.6250

Epoch 1500, Training loss: 26740831012479.1875, Validation loss: 25039409951772.1055

Epoch 2000, Training loss: 26700890961250.7227, Validation loss: 25027422529232.8438

Epoch 2500, Training loss: 26652257436582.1211, Validation loss: 25012518528600.2812

Epoch 3000, Training loss: 26595772699572.8594, Validation loss: 24994623487049.5469

Epoch 3500, Training loss: 26533310861371.6367, Validation loss: 24973802525837.6055

Epoch 4000, Training loss: 26468207913931.1953, Validation loss: 24950347678803.8438

Epoch 4500, Training loss: 26405553343092.6289, Validation lo

tensor([-0.5508,  1.4538,  4.9451,  8.8884, 16.0422, 21.6918, 31.7022, 36.5956,
        46.0317, 48.1953, 51.9843, 51.9644, 50.0357], requires_grad=True)

In [55]:
#Adam Number 3 learning rate of 0.001
learning_rate_Adam3 = 0.001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
optimizer_Adam3 = optim.Adam([params], lr=learning_rate_Adam3)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_Adam3,
    params = params,
    t_u_train = x3_train_ten,
    t_c_train = y3_train_ten,
    t_u_val = x3_val_ten,
    t_c_val = y3_val_ten,
)

Epoch 1, Training loss: 296683442882877.3750, Validation loss: 54364782483269.1953

Epoch 2, Training loss: 296130908817249.7500, Validation loss: 54302282830766.4062

Epoch 3, Training loss: 295578939957657.1250, Validation loss: 54239849893038.2422

Epoch 500, Training loss: 109886227377696.1406, Validation loss: 33497140310324.9023

Epoch 1000, Training loss: 44739907168983.4375, Validation loss: 26592161288247.7695

Epoch 1500, Training loss: 29527009579545.6133, Validation loss: 25201206189952.6172

Epoch 2000, Training loss: 27143355164035.1992, Validation loss: 25062928282151.1016

Epoch 2500, Training loss: 26833097073016.8086, Validation loss: 25057008484403.5664

Epoch 3000, Training loss: 26799493805515.1445, Validation loss: 25056312032467.1641

Epoch 3500, Training loss: 26795040264333.4531, Validation loss: 25055509003589.0312

Epoch 4000, Training loss: 26791481357469.0039, Validation loss: 25054490526073.2148

Epoch 4500, Training loss: 26786997494854.1758, Validation l

tensor([0.0432, 0.1065, 0.2139, 0.3464, 0.6122, 0.8993, 1.6139, 2.1694, 3.7531,
        4.4223, 6.6024, 6.8773, 5.0223], requires_grad=True)

In [56]:
#Adam Number 4 learning rate of 0.0001
learning_rate_Adam4 = 0.0001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
optimizer_Adam4 = optim.Adam([params], lr=learning_rate_Adam4)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer_Adam4,
    params = params,
    t_u_train = x3_train_ten,
    t_c_train = y3_train_ten,
    t_u_val = x3_val_ten,
    t_c_val = y3_val_ten,
)

Epoch 1, Training loss: 296683442882877.3750, Validation loss: 54364782483269.1953

Epoch 2, Training loss: 296628154157794.2500, Validation loss: 54358528397781.7578

Epoch 3, Training loss: 296572871086792.8750, Validation loss: 54352274979680.7188

Epoch 500, Training loss: 270109003760856.7500, Validation loss: 51362140927160.4375

Epoch 1000, Training loss: 245434834078824.0312, Validation loss: 48580926983743.0312

Epoch 1500, Training loss: 222548239847675.9688, Validation loss: 46007989507670.1719

Epoch 2000, Training loss: 201309678970114.3125, Validation loss: 43627219788308.9219

Epoch 2500, Training loss: 181605828994504.0312, Validation loss: 41425533753622.7656

Epoch 3000, Training loss: 163345973360442.3750, Validation loss: 39392443444055.6953

Epoch 3500, Training loss: 146457860155213.8125, Validation loss: 37519571250265.8438

Epoch 4000, Training loss: 130883923243264.1562, Validation loss: 35800206377602.6797

Epoch 4500, Training loss: 116577281286144.0000, Vali

tensor([0.5443, 0.5445, 0.5447, 0.5450, 0.5456, 0.5463, 0.5482, 0.5502, 0.5617,
        0.5738, 1.5656, 1.5788, 0.5013], requires_grad=True)